In [1]:
from pyspark.sql import *

In [2]:
spark = SparkSession.builder.appName("yelp").getOrCreate()

In [3]:
biz = spark.read.json("yelp_business.json")

In [4]:
biz.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [7]:
biz.show(2)

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|Santa Barbara|                null|      0|34.4266787|-119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|       Affton|{8:0-18:30, 0:0-0...|      1| 38.551126|  -90.335695|    

In [8]:
biz.createOrReplaceTempView("biz")

In [14]:
spark.sql("select count(1) as business from biz").show()

+--------+
|business|
+--------+
|  150346|
+--------+



In [16]:
spark.sql("select state, count(1) as business from biz group by state ORDER by business DESC").show(5)

+-----+--------+
|state|business|
+-----+--------+
|   PA|   34039|
|   FL|   26330|
|   TN|   12056|
|   IN|   11247|
|   MO|   10913|
+-----+--------+
only showing top 5 rows



In [12]:
spark.sql("select state, ROUND(AVG(review_count)) as avg_reviews \
          from biz group by state order by avg_reviews DESC LIMIT 5").show(5)

+-----+-----------+
|state|avg_reviews|
+-----+-----------+
|   LA|       75.0|
|   CA|       65.0|
|   NV|       53.0|
|   TN|       50.0|
|   PA|       45.0|
+-----+-----------+



In [33]:
spark.sql("Select * from biz where hours.Wednesday = '10:0-19:0' LIMIT 2").show()

+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+-------------+--------------+--------------+-----------+------------+-----+-----+
|           address|          attributes|         business_id|          categories|       city|               hours|is_open|     latitude|     longitude|          name|postal_code|review_count|stars|state|
+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+-------------+--------------+--------------+-----------+------------+-----+-----+
|      712 Adams St|{null, null, null...|M0XSSHqrASOnhgbWD...|Vape Shops, Tobac...|New Orleans|{10:0-19:0, 10:0-...|      1|29.9414679565| -90.129952757|Herb Import Co|      70118|           5|  4.0|   LA|
|30 W Lancaster Ave|{null, null, null...|8sshLb4UU7emeUDvt...|Dance Wear, Sport...|      Paoli|{10:0-19:0, 0:0-0...|      1|40.0415845065|-75.4849526577|     DanceLine|      19